In [1]:
import pandas as pd

In [2]:
# prompt: mount with google drive

import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
data=pd.read_csv("/content/drive/MyDrive/final_data.csv")

,text,emotion,cleaned_text,embedding,emotion_Anger,emotion_Anxiety,emotion_Curiosity,emotion_Desire,emotion_Happiness,emotion_Love & Care,emotion_Neutral,emotion_Relief,emotion_Sadness
0,I am getting a lot of weird replies...,Neutral,getting lot weird replies,[[ 3.57603505e-02 1.88769430e-01 -5.63014112e...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Why is it illegal?,Neutral,illegal,[[-3.13512266e-01 5.34555972e-01 -5.06467640e...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,[NAME] just think of how overworked her liver ...,Happiness,name think overworked liver kidneys must,[[-9.83442590e-02 3.89946789e-01 -4.01094072e...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,But doing those things makes me feel so.... icky.,Sadness,things makes feel icky,[[ 4.80609946e-02 3.33314270e-01 1.51030645e...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Shame on you,Anger,shame,[[-8.55700299e-02 2.50925511e-01 -1.43386796e...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57116,"Ha, that would be a novel experience. He'd bet...",Desire,ha would novel experience better let deal,[[-2.79613025e-02 1.31248564e-01 -1.78521685e...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
57117,Very sad. People like this are ruining Aus.,Sadness,sad people like ruining aus,[[ 1.09247044e-01 3.23958039e-01 -1.02081627e...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
57118,"It was worth mentioning anyways, the Packers a...",Neutral,worth mentioning anyways packers special kind ...,[[ 1.67145580e-02 6.16340339e-02 -1.64117783e...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
57119,They want to make gun ownership so expensive a...,Anger,want make gun ownership expensive onerous peop...,[[ 2.04446822e-01 3.60814929e-01 2.40353718e...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
X_experiment = data.iloc[:, :-9].values  # First 768 columns are BERT embeddings
Y_experiment = data.iloc[:, -9:].values  # Last 10 columns are emotion labels


from sklearn.model_selection import train_test_split
X_train_experiment, X_test_experiment, Y_train_experiment, Y_test_experiment = train_test_split(X_experiment, Y_experiment, test_size=0.2, random_state=42)


In [6]:
# Assuming you have X_train, X_test, y_train, y_test
X_train_experiment = X_train_experiment.reshape(X_train_experiment.shape[0], X_train_experiment.shape[1], 1)  # (45696, 768, 1)
X_test_experiment = X_test_experiment.reshape(X_test_experiment.shape[0], X_test_experiment.shape[1], 1)      # (11425, 768, 1)

print("New Shapes:")
print("X_train:", X_train_experiment.shape)  # (45696, 768, 1)
print("y_train:", Y_train_experiment.shape)  # (45696, 9)
print("X_test:", X_test_experiment.shape)    # (11425, 768, 1)
print("y_test:", Y_test_experiment.shape)    # (11425, 9)


New Shapes:
X_train: (45696, 768, 1)
y_train: (45696, 9)
X_test: (11425, 768, 1)
y_test: (11425, 9)


In [ ]:
# Assuming you have X_train, X_test, y_train, y_test
X_train_experiment = X_train_experiment.reshape(X_train_experiment.shape[0], X_train_experiment.shape[1], 1)  # (45696, 768, 1)
X_test_experiment = X_test_experiment.reshape(X_test_experiment.shape[0], X_test_experiment.shape[1], 1)      # (11425, 768, 1)

print("New Shapes:")
print("X_train:", X_train_experiment.shape)  # (45696, 768, 1)
print("y_train:", Y_train_experiment.shape)  # (45696, 9)
print("X_test:", X_test_experiment.shape)    # (11425, 768, 1)
print("y_test:", Y_test_experiment.shape)    # (11425, 9)


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, LSTM, Input



In [9]:
model = Sequential([
    Input(shape=(X_train_experiment.shape[1], 1)),  # Input shape (num_features, 1) for LSTM
    Bidirectional(LSTM(128, return_sequences=True)),  # First BiLSTM layer
    Dropout(0.3),
    Bidirectional(LSTM(64)),  # Second BiLSTM layer
    Dropout(0.3),
    Dense(64, activation='relu'),  # Fully connected layer
    Dropout(0.3),
    Dense(Y_train_experiment.shape[1], activation='softmax')  # Output layer (multi-class classification)
])

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_bilstm_model.h5', monitor='val_loss', save_best_only=True)


In [12]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 768, 256)            │         133,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 768, 256)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 9)                   │             585 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 306,313 (1.17 MB)

 Trainable params: 306,313 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
history = model.fit(
    X_train_experiment.reshape(-1, X_train_experiment.shape[1], 1), Y_train_experiment,
    validation_data=(X_test_experiment.reshape(-1, X_test_experiment.shape[1], 1), Y_test_experiment),
    epochs=30, batch_size=32,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.1350 - loss: 2.1742

1428/1428 ━━━━━━━━━━━━━━━━━━━━ 157s 106ms/step - accuracy: 0.1350 - loss: 2.1742 - val_accuracy: 0.1599 - val_loss: 2.1515
Epoch 2/30
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.1646 - loss: 2.1520

1428/1428 ━━━━━━━━━━━━━━━━━━━━ 205s 109ms/step - accuracy: 0.1646 - loss: 2.1520 - val_accuracy: 0.1675 - val_loss: 2.1396
Epoch 3/30
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.1757 - loss: 2.1367

1428/1428 ━━━━━━━━━━━━━━━━━━━━ 150s 105ms/step - accuracy: 0.1757 - loss: 2.1367 - val_accuracy: 0.1898 - val_loss: 2.1178
Epoch 4/30
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.1824 - loss: 2.1303

1428/1428 ━━━━━━━━━━━━━━━━━━━━ 209s 110ms/step - accuracy: 0.1824 - loss: 2.1303 - val_accuracy: 0.1980 - val_loss: 2.1048
Epoch 5/30
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.1898 - loss: 2.1163

...

1428/1428 ━━━━━━━━━━━━━━━━━━━━ 209s 109ms/step - accuracy: 0.2371 - loss: 2.0210 - val_accuracy: 0.2371 - val_loss: 2.0254
Epoch 16/30
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.2455 - loss: 2.0177

1428/1428 ━━━━━━━━━━━━━━━━━━━━ 201s 109ms/step - accuracy: 0.2455 - loss: 2.0177 - val_accuracy: 0.2432 - val_loss: 2.0159
Epoch 17/30
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 202s 110ms/step - accuracy: 0.2450 - loss: 2.0123 - val_accuracy: 0.2411 - val_loss: 2.0215
Epoch 18/30
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 201s 109ms/step - accuracy: 0.2482 - loss: 2.0065 - val_accuracy: 0.2353 - val_loss: 2.0225
Epoch 19/30
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.2508 - loss: 1.9965

1428/1428 ━━━━━━━━━━━━━━━━━━━━ 203s 110ms/step - accuracy: 0.2508 - loss: 1.9965 - val_accuracy: 0.2439 - val_loss: 2.0130
Epoch 20/30
1428/1428 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.2542 - loss: 1.9935

1428/1428 ━━━━━━━━━━━━━━━━━━━━ 201s 109ms/step - accuracy: 0.2542 - loss: 1.9935 - val_accuracy: 0.2456 - val_loss: 2.0129
Epoch 21/30


KeyboardInterrupt: 